# Project 2 — Subscriptions Churn Analysis

In [ ]:
import pandas as pd
import numpy as np
customers = pd.read_csv("../data_raw/customers.csv", parse_dates=["signup_date"])
subscriptions = pd.read_csv("../data_raw/subscriptions.csv", parse_dates=["period_start","period_end"])
usage = pd.read_csv("../data_raw/usage.csv", parse_dates=["month"])
tickets = pd.read_csv("../data_raw/tickets.csv", parse_dates=["opened_at","closed_at"])
surveys = pd.read_csv("../data_raw/surveys.csv", parse_dates=["survey_date"])
customers.head()

## Dataset analitic lunar

In [ ]:
active = (subscriptions[subscriptions["is_active"]==1]
          .assign(month=subscriptions["period_start"].dt.to_period("M"))
          .groupby(["customer_id","month"]).size().reset_index(name="active"))
active["active"]=1
u = (usage.assign(month=usage["month"].astype('period[M]'))
          .groupby(["customer_id","month"]).agg(events=("events","sum"),
                                               minutes=("minutes_used","sum"),
                                               featX=("feature_X_used","sum")).reset_index())
t = (tickets.assign(month=tickets["opened_at"].dt.to_period("M"),
                    duration_hours=(tickets["closed_at"]-tickets["opened_at"]).dt.total_seconds()/3600)
            .groupby(["customer_id","month"])
            .agg(tickets=("ticket_id","nunique"),
                 avg_close_h=("duration_hours","mean")).reset_index())
df = (active.merge(u, on=["customer_id","month"], how="left")
            .merge(t, on=["customer_id","month"], how="left")
            .fillna({"events":0,"minutes":0,"featX":0,"tickets":0,"avg_close_h":0}))
df.head()

## Etichetă churn în t+1 și t-test

In [ ]:
next_active = active[["customer_id","month"]].rename(columns={"month":"month_next"})
df2 = df.copy()
df2["month_next"]=df2["month"]+1
df2 = df2.merge(next_active.assign(active_next=1), on=["customer_id","month_next"], how="left")
df2["churn"] = (df2["active_next"].isna()).astype(int)
from scipy.stats import ttest_ind
t_stat, p_val = ttest_ind(df2.loc[df2.churn==1,"minutes"],
                          df2.loc[df2.churn==0,"minutes"],
                          equal_var=False, nan_policy='omit')
p_val